In [1]:
import pandas as pd
import pathlib
import re
import os
import numpy as np

In [2]:
annotations_path = pathlib.Path("idr0013-screenA-annotation.csv.gz")
annotations = pd.read_csv(annotations_path, compression='gzip', dtype=object)
annotations

,Plate,Well Number,Well,Plate_Well,Characteristics [Organism],Term Source 1 REF,Term Source 1 Accession,Characteristics [Cell Line],Term Source 2 REF,Term Source 2 Accession,...,Phenotype 15 Term Accession,Phenotype 16,Phenotype 16 Term Name,Phenotype 16 Term Accession,Phenotype 17,Phenotype 17 Term Name,Phenotype 17 Term Accession,Phenotype 18,Phenotype 18 Term Name,Phenotype 18 Term Accession
0,LT0001_02,1,A1,LT0001_02_A1,Homo sapiens,NCBITaxon,NCBITaxon_9606,HeLa,EFO,EFO_0001185,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LT0001_02,2,A2,LT0001_02_A2,Homo sapiens,NCBITaxon,NCBITaxon_9606,HeLa,EFO,EFO_0001185,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,LT0001_02,3,A3,LT0001_02_A3,Homo sapiens,NCBITaxon,NCBITaxon_9606,HeLa,EFO,EFO_0001185,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,LT0001_02,4,A4,LT0001_02_A4,Homo sapiens,NCBITaxon,NCBITaxon_9606,HeLa,EFO,EFO_0001185,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,LT0001_02,5,A5,LT0001_02_A5,Homo sapiens,NCBITaxon,NCBITaxon_9606,HeLa,EFO,EFO_0001185,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196603,LT0603_06,380,P20,LT0603_06_P20,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196604,LT0603_06,381,P21,LT0603_06_P21,Homo sapiens,NCBITaxon,NCBITaxon_9606,HeLa,EFO,EFO_0001185,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196605,LT0603_06,382,P22,LT0603_06_P22,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196606,LT0603_06,383,P23,LT0603_06_P23,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# remove annotations for plates that are missing
annotations = annotations.loc[annotations['Plate Issues'] != "plate missing"]

In [4]:
control_annotations = annotations.loc[annotations['Control Type'].notnull()]
control_annotations = control_annotations.loc[control_annotations['Control Type'] != "empty well"]
control_annotations['Control Type'].value_counts()

negative control    3853
positive control    3374
Name: Control Type, dtype: int64

In [5]:
control_metadata = control_annotations[["Plate", "Well"]]
# add constant frame to all wells
# frames need to be string in format "x,y,z"
control_metadata = control_metadata.assign(Frames="50")

control_metadata['Well Number'] = control_annotations['Well Number']
control_metadata['Control Type'] = control_annotations['Control Type']
control_metadata['Original Gene Target'] = control_annotations['Original Gene Target']
control_metadata['Original Gene Target'] = control_metadata['Original Gene Target'].replace(np.nan, "negative control")

control_metadata["Plate_Map_Name"] = control_metadata['Plate'].astype(str) +"_"+ control_metadata["Well Number"]
control_metadata["Gene_Replicate"] = 1
control_metadata["Site"] = 1
control_metadata["DNA"] = control_metadata['Plate'].astype(str) +"/"+ control_metadata['Plate'].astype(str) +"_"+ control_metadata["Well Number"] +"_"+ control_metadata["Frames"] + ".tif"

# remove A1 wells because of irregular illumination
control_metadata = control_metadata[control_metadata["Well"] != "A1"]
control_metadata = control_metadata.reset_index(drop=True)

control_metadata

,Plate,Well,Frames,Well Number,Control Type,Original Gene Target,Plate_Map_Name,Gene_Replicate,Site,DNA
0,LT0001_02,A4,50,4,positive control,ENSG00000149503,LT0001_02_4,1,1,LT0001_02/LT0001_02_4_50.tif
1,LT0001_02,A15,50,15,negative control,negative control,LT0001_02_15,1,1,LT0001_02/LT0001_02_15_50.tif
2,LT0001_02,B2,50,26,negative control,negative control,LT0001_02_26,1,1,LT0001_02/LT0001_02_26_50.tif
3,LT0001_02,C1,50,49,positive control,ENSG00000149503,LT0001_02_49,1,1,LT0001_02/LT0001_02_49_50.tif
4,LT0001_02,C4,50,52,positive control,ENSG00000149503,LT0001_02_52,1,1,LT0001_02/LT0001_02_52_50.tif
...,...,...,...,...,...,...,...,...,...,...
6738,LT0603_06,O13,50,349,positive control,KIF11,LT0603_06_349,1,1,LT0603_06/LT0603_06_349_50.tif
6739,LT0603_06,O16,50,352,negative control,negative control,LT0603_06_352,1,1,LT0603_06/LT0603_06_352_50.tif
6740,LT0603_06,P3,50,363,negative control,negative control,LT0603_06_363,1,1,LT0603_06/LT0603_06_363_50.tif
6741,LT0603_06,P21,50,381,positive control,COPB,LT0603_06_381,1,1,LT0603_06/LT0603_06_381_50.tif


In [6]:
control_metadata['Control Type'].value_counts()

negative control    3853
positive control    2890
Name: Control Type, dtype: int64

In [7]:
metadata_save_path = pathlib.Path("data_to_process.tsv")
control_metadata.to_csv(metadata_save_path, sep="\t")